# Liquidity Stress Tool

Interactive liquidity stress testing tool used to assess whether a fund can meet redemptions under market stress.

The model operates on aggregated liquidity buckets, mirroring how liquidity risk teams perform funding analysis.


In [1]:
import sys
from pathlib import Path

project_root = Path.cwd().parent
sys.path.append(str(project_root))

In [2]:
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

from engine.assets import default_liquidity_profile
from engine.liquidity import run_waterfall, apply_stress
from engine.metrics import liquidity_metrics
from engine.assumptions import ASSUMPTIONS
from engine.scenario_translation import translate_scenario


In [3]:
# Helpers
def fmt_bn(x):
    return f"${x / 1e9:,.2f} bn"

def stress_summary(metrics):
    status = "BREACH" if metrics["breach"] else "PASS"
    print("\n--- Stress Summary ---")
    print(f"Result: {status}")
    print(f"Liquidity Coverage: {metrics['liquidity_coverage']:.2f}")
    print(f"Days to Liquidity: {metrics['days_to_liquidity']}")


def plot_liquidity_charts(metrics, waterfall):
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))

    # ---- Liquidity Coverage ----
    coverage_vals = [
        metrics["cash_required"] / 1e9,
        metrics["cash_raised"] / 1e9
    ]
    axes[0].bar(["Cash Required", "Cash Raised"], coverage_vals)
    axes[0].set_ylabel("CAD (bn)")
    axes[0].set_title("Liquidity Coverage")

    # ---- Liquidity Waterfall ----
    df = waterfall[waterfall["cash_used"] > 0]
    if not df.empty:
        axes[1].barh(df["bucket"], df["cash_used"] / 1e9)
        axes[1].set_xlabel("Cash Raised (CAD bn)")
        axes[1].set_title("Liquidity Waterfall")

    plt.tight_layout()
    plt.show()

In [ ]:
# sliders
redemption_slider = widgets.FloatSlider(
    value=0.0,
    min=0.0,
    max=0.99,
    step=0.01,
    description="Redemption %",
    readout_format=".0%"
)

equity_slider = widgets.FloatSlider(
    value=0.0,
    min=0.0,
    max=0.99,
    step=0.01,
    description="Equity Drawdown",
    readout_format=".0%"
)

# checkboxes
freeze_cash = widgets.Checkbox(value = False, description = "Freeze Cash & ST Bonds")
freeze_t1 = widgets.Checkbox(value = False, description="Freeze Public Equities [T+1]")
freeze_t5 = widgets.Checkbox(value = False, description = "Freeze Public Credit [T+5]")
freeze_t30 = widgets.Checkbox(value = False, description = "Freeze Real Estate [T+30]")
freeze_t90 = widgets.Checkbox(value = False, description = "Freeze Infrastructure [T+90]")

# full controls
controls = widgets.VBox([equity_slider, redemption_slider, freeze_cash, freeze_t1, freeze_t5, freeze_t30, freeze_t90])

display(controls)

In [ ]:
output = widgets.Output()
display(output)

Output()

In [10]:
def run_stress(_=None):
    with output:
        clear_output()

        # Base portfolio (CAD dollars)
        profile = default_liquidity_profile()

        # Apply liquidity freezes
        if freeze_cash.value:
            profile.loc[
                profile["bucket"] == "Cash & Short-Term Bonds",
                "available",
            ] = False

        if freeze_t1.value:
            profile.loc[
                profile["bucket"] == "Public Equities [T+1]",
                "available",
            ] = False

        if freeze_t5.value:
            profile.loc[
                profile["bucket"] == "Public Credit [T+5]",
                "available",
            ] = False

        if freeze_t30.value:
            profile.loc[
                profile["bucket"] == "Real Estate [T+30]",
                "available",
            ] = False

        if freeze_t90.value:
            profile.loc[
                profile["bucket"] == "Infrastructure [T+90]",
                "available",
            ] = False

        # Scenario
        scenario = {
            "equity_drawdown": -equity_slider.value,
            "redemption": redemption_slider.value,
        }

        # Translate scenario
        stressed_profile, liquidity_demand = translate_scenario(
            profile=profile,
            scenario=scenario,
            assumptions=ASSUMPTIONS,
        )

        # Apply valuation stress
        stressed_profile = apply_stress(stressed_profile)

        # Run liquidity waterfall
        waterfall, summary = run_waterfall(
            stressed_profile=stressed_profile,
            cash_required=liquidity_demand,
        )

        metrics = liquidity_metrics(summary)

        stress_summary(metrics)
        print(f"\nScenario: {scenario}")
        print(f"Total Fund Value: {fmt_bn(profile['market_value'].sum())}")

        print("\nLiquidity Waterfall (Table)")
        waterfall_display = waterfall.copy()
        for col in ["stressed_value", "cash_used", "remaining_value"]:
            waterfall_display[col] = waterfall_display[col].apply(fmt_bn)

        display(waterfall_display)

        plot_liquidity_charts(metrics, waterfall)


In [11]:
equity_slider.observe(run_stress, names="value")
redemption_slider.observe(run_stress, names="value")
freeze_cash.observe(run_stress, names="value")
freeze_t1.observe(run_stress, names="value")
freeze_t5.observe(run_stress, names="value")
freeze_t30.observe(run_stress, names="value")
freeze_t90.observe(run_stress, names="value")

run_stress()